In [57]:
import numpy as np 
import pandas as pd 
from numpy.linalg import eig

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [107]:

hashtag_trump = pd.read_csv("./datasets/hashtag_donaldtrump.csv",engine ="python")
hashtag_biden = pd.read_csv("./datasets/hashtag_joebiden.csv",engine ="python")
biden_tweets = pd.read_csv("./datasets/tweets_of_joe_biden.csv",engine ="python")
trump_tweets = pd.read_json("./datasets/trump-tweets.json")

In [109]:
trump= trump_tweets.copy()


In [117]:
trump_deleted = pd.read_json("./trump_deleted.json")
trump_deleted
status = list()
id_deleted = list(trump_deleted['id'])
print(id_deleted)
for ids in trump['id']:
    if(ids in id_deleted):
        status.insert(ids,"Deleted")
    else:
        status.insert(ids,"Normal")
status
trump['status'] = status
trump


[1324445863083708416, 1324012948965261312, 1323863787825438720, 1323714481353068544, 1323692146147340288, 1321965155509391360, 1321195996605394944, 1320226451166253056, 1319494198039441408, 1317497136523759616, 1317058851003420672, 1316711680919908352, 1316594633548484608, 1316579572603904000, 1316250377415790592, 1316223657908240384, 1315964956601446400, 1315872126327500800, 1315871989907755008, 1315871867899645952, 1315871590685564928, 1315871589129498624, 1315868183459508224, 1315836801354027008, 1315286705696579584, 1314278414359113728, 1313852035087953920]


,date,favorites,id,isRetweet,retweets,text,status
0,2020-11-06 23:25:39,580972,1324855496722026496,False,101589,I had such a big lead in all of these states l...,Normal
1,2020-11-06 22:50:13,677422,1324846580147642368,False,116319,Joe Biden should not wrongfully claim the offi...,Normal
2,2020-11-06 17:38:17,652476,1324768077939118080,False,119892,Where are the missing military ballots in Geor...,Normal
3,2020-11-06 16:37:01,416028,1324752659996397568,False,68191,With the attack by the Radical Left Dems on th...,Normal
4,2020-11-06 16:28:42,156526,1324750568817102848,False,28012,....has been able to render decisions in a mat...,Normal
...,...,...,...,...,...,...,...
1515,2020-10-07 00:29:24,0,1313637515065204736,True,12897,RT @paulsperry_: BREAKING: An FBI investigator...,Normal
1516,2020-10-07 00:29:06,0,1313637441006383104,True,12288,RT @paulsperry_: BREAKING: Records show Obama'...,Normal
1517,2020-10-06 23:59:57,89191,1313630105831383040,False,20366,Republicans need to get smart and confirm Nate...,Normal
1518,2020-10-06 23:36:46,182368,1313624267842560000,False,46938,Crazy Nancy Pelosi and the Radical Left Democr...,Normal


In [118]:
#We now need to convert all the "favorites" into integers as some are integers and some strings. Pandas might automaticall do this for us
trump = trump.astype({'favorites': 'int'})
trump = trump.astype({'retweets': 'int'})
trump.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       1520 non-null   datetime64[ns]
 1   favorites  1520 non-null   int32         
 2   id         1520 non-null   int64         
 3   isRetweet  1520 non-null   bool          
 4   retweets   1520 non-null   int32         
 5   text       1520 non-null   object        
 6   status     1520 non-null   object        
dtypes: bool(1), datetime64[ns](1), int32(2), int64(1), object(2)
memory usage: 61.0+ KB


In [121]:

# Here we have created a trump data set which displays all the deleted tweets. We do this by cross correlating trumps deleted tweets with the list of all his tweets
#The retweets have 0 favourites (which are basically likes) and have an RT at the front 
# To distinquish between the tweets trump deleted and the tweets deleted by twitter (restricted), we make the assumption that whenever a tweet is deleted by twitter the 
# like and retweet numbers are set to 0. We use this to distrinquish 
new_status = list()
for index,val in enumerate(trump['status']):
    list_fav = list(trump['favorites'])
    list_retweets = list(trump['retweets'])
    if(list_fav[index]==0 and list_retweets[index] == 0):
        new_status.insert(index, "Restricted")
    else:
        new_status.insert(index,val)
trump['status'] = new_status
trump[trump['status'] == "Normal"] 



            

,date,favorites,id,isRetweet,retweets,text,status
0,2020-11-06 23:25:39,580972,1324855496722026496,False,101589,I had such a big lead in all of these states l...,Normal
1,2020-11-06 22:50:13,677422,1324846580147642368,False,116319,Joe Biden should not wrongfully claim the offi...,Normal
2,2020-11-06 17:38:17,652476,1324768077939118080,False,119892,Where are the missing military ballots in Geor...,Normal
3,2020-11-06 16:37:01,416028,1324752659996397568,False,68191,With the attack by the Radical Left Dems on th...,Normal
4,2020-11-06 16:28:42,156526,1324750568817102848,False,28012,....has been able to render decisions in a mat...,Normal
...,...,...,...,...,...,...,...
1515,2020-10-07 00:29:24,0,1313637515065204736,True,12897,RT @paulsperry_: BREAKING: An FBI investigator...,Normal
1516,2020-10-07 00:29:06,0,1313637441006383104,True,12288,RT @paulsperry_: BREAKING: Records show Obama'...,Normal
1517,2020-10-06 23:59:57,89191,1313630105831383040,False,20366,Republicans need to get smart and confirm Nate...,Normal
1518,2020-10-06 23:36:46,182368,1313624267842560000,False,46938,Crazy Nancy Pelosi and the Radical Left Democr...,Normal


The trump data now has a status column which says if the tweet if the tweet is "normal", "Resitrcited" or "deleted".We have also converted all the 'favorites' and the 'retweets' into integers. 